# Test Stable Diffusion with text-to-image


In [ ]:
!pip install --upgrade torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install transformers pandas numpy tqdm wandb

In [ ]:
import torch
import torchvision
# import torchaudio

print(f'torch version: {torch.__version__}')
print(f'torchvision version: {torchvision.__version__}')
# print(f'torchaudio version: {torchaudio.__version__}')

In [ ]:
import subprocess
import os 


def git(*args):
    result = subprocess.run(['git'] + list(args), check=True)
    return result

os.chdir('/kaggle/working/')
repo_url = "https://github.com/HoangTran223/SD_from_scratch.git"

git("clone", repo_url)

In [ ]:
!pip install pipeline

In [ ]:
os.chdir('/kaggle/working/SD_from_scratch/')
print("After working directory:", os.getcwd())
!ls /kaggle/working/SD_from_scratch/

In [ ]:
if torch.cuda.is_available():
    DEVICE = "cuda"
else:
    DEVICE = "cpu"
print(f"Using device: {DEVICE}")

In [ ]:
from huggingface_hub import hf_hub_download
os.makedirs("/kaggle/working/data", exist_ok=True)

import requests

url = "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt"
output_path = "/kaggle/working/data/v1-5-pruned-emaonly.ckpt"

response = requests.get(url)
with open(output_path, "wb") as f:
    f.write(response.content)

In [ ]:
from diffusers import DiffusionPipeline
pipe = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5")

In [ ]:
pipe.tokenizer.save_pretrained("/kaggle/working/data")
# pipe.save_pretrained("/kaggle/working/data")

In [ ]:
!git stash
!git pull origin main
# import sys
# sys.path.append("/kaggle/working/SD_from_scratch")

In [ ]:
from sd import model_loader, pipeline 
import torch
import numpy
from transformers import CLIPTokenizer

tokenizer = CLIPTokenizer(
    vocab_file="/kaggle/working/data/vocab.json",
    merges_file="/kaggle/working/data/merges.txt"
)

In [ ]:
model_path_1 = "/kaggle/working/data/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_path_1, DEVICE)

In [ ]:
prompt = "A girl student, best quality, vibrant colors, detailed anime style, glasses, highly detailed, ultra sharp, cinematic, school uniform, 8k resolution, 3D illustration."
negative_prompt = "lowres, bad anatomy, extra fingers, watermark, text, blurry, distorted, sketch."

In [ ]:
output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=negative_prompt,
    input_image=None,
    strength=0.0,
    do_cfg=True,
    cfg_scale=8,
    sampler_name=sampler,
    n_inference_steps=50,
    seed=42,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
)

Image.fromarray(output_image)